In [1]:
import numpy as np
import math
import roboticstoolbox as rtb
from spatialmath.base import *
from spatialmath import SE3
import spatialmath.base.symbolic as sym

%matplotlib notebook

We use the Spatial Math Toolbox wrapper of SymPy which is `spatialmath.base.symbolic` which is imported above.

## Creating a symbolic variable
We will first create a symbolic variable

In [2]:
theta = sym.symbol('theta')

In [3]:
theta

theta

which displays as a Greek letter, it has the type

In [4]:
type(theta)

sympy.core.symbol.Symbol

The function specifies that the symbolic variables are real valued by default, which will simplify subsequent simplification steps

In [5]:
theta.is_real

True

We can test if a variable is symbolic

In [6]:
sym.issymbol(theta)

True

In [7]:
sym.issymbol(3.7)

False

## Symbolics with the Spatial Math Toolbox
Many Spatial Toolbox functions handle symbolic variables

In [8]:
R = rot2(theta)
R

array([[cos(theta), -sin(theta)],
       [sin(theta), cos(theta)]], dtype=object)

and return a NumPy array with symbolic values.  

The 3D case is similar

In [9]:
R = rotx(theta)
R

array([[1, 0, 0],
       [0, cos(theta), -sin(theta)],
       [0, sin(theta), cos(theta)]], dtype=object)

In [10]:
T = trotx(theta)
T

array([[1, 0, 0, 0],
       [0, cos(theta), -sin(theta), 0],
       [0, sin(theta), cos(theta), 0],
       [0, 0, 0, 1]], dtype=object)

The elements of this NumPy array are all objects, as indicated by the `dtype`.  However when we index the elements, they will be converted back to numeric types if possible

In [11]:
type(T[0,0])

int

In [12]:
type(T[1,1])

cos

We can perform arithmetic on such matrices, for example

In [13]:
T @ T

array([[1, 0, 0, 0],
       [0, -sin(theta)**2 + cos(theta)**2, -2*sin(theta)*cos(theta), 0],
       [0, 2*sin(theta)*cos(theta), -sin(theta)**2 + cos(theta)**2, 0],
       [0, 0, 0, 1]], dtype=object)

## Pose classes
The symbolic capability extends to the pose classes

In [14]:
T = SE3.Rx(theta)
T

SE3:   1           0           0           0            
       0          cos(theta)  -sin(theta)  0            
       0          sin(theta)  cos(theta)   0            
       0           0           0           1            
    

In [15]:
T2 = T * T
T2

SE3:   1          0           0            0            
      0           -sin(theta)**2 + cos(theta)**2-2*sin(theta)*cos(theta)0             
      0           2*sin(theta)*cos(theta)-sin(theta)**2 + cos(theta)**20             
       0          0           0            1            
    

but the colored layout is problematic.

## Robot forward kinematics

We will create a symbolic version of the robot model

In [16]:
puma = rtb.models.DH.Puma560(symbolic=True)

In [17]:
print(puma)

┏━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃θⱼ ┃   dⱼ    ┃   aⱼ   ┃  ⍺ⱼ   ┃
┣━━━╋━━━━━━━━━╋━━━━━━━━╋━━━━━━━┫
┃q1 ┃   0.672 ┃      0 ┃  pi/2 ┃
┃q2 ┃       0 ┃ 0.4318 ┃     0 ┃
┃q3 ┃ 0.15005 ┃ 0.0203 ┃ -pi/2 ┃
┃q4 ┃  0.4318 ┃      0 ┃  pi/2 ┃
┃q5 ┃       0 ┃      0 ┃ -pi/2 ┃
┃q6 ┃       0 ┃      0 ┃     0 ┃
┗━━━┻━━━━━━━━━┻━━━━━━━━┻━━━━━━━┛

┌───┬────────────────────────────┐
│qz │ 0°, 0°, 0°, 0°, 0°, 0°     │
│qr │ 0°, 90°, -90°, 0°, 0°, 0°  │
│qs │ 0°, 0°, -90°, 0°, 0°, 0°   │
│qn │ 0°, 45°, 180°, 0°, 45°, 0° │
└───┴────────────────────────────┘



We see that the $\alpha_j$ values are now given in radians and are colored red.  This means the value is a symbolic expression, for the first link it is $\pi/2$ which is a precise value, compared to the non-symbolic case which is a floating point number 1.5707963267948966 that only  approximates $\pi/2$.

The next thing we need to do is create a vector of joint coordinates which are all symbols

In [18]:
q = sym.symbol('q_:6')
q

(q_0, q_1, q_2, q_3, q_4, q_5)

We use the underscore, because the value of the symbol is pretty printed by SymPy as a subscript (just as it is with LaTeX math notation)

In [19]:
q[0]

q_0

We are now set to compute the forward kinematics which will be a matrix whose elements will be complicated expressions of symbolic joint variables and kinematic parameters

In [20]:
T = puma.fkine(q)
T

SE3:  (((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5)-(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5)-((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4)0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)  
      (((-sin(q_0)*sin(q_1)*sin(q_2) + sin(q_0)*cos(q_1)*cos(q_2))*cos(q

The color coding helps us identify the rotational and translational parts, but the format is not very readable.  We can display the underlying NumPy array

In [21]:
T.A

array([[(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5),
        -(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5),
        -((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4),
        0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)],
       [(((-sin(q_0)*sin(q_1)*sin(q_2) +

which is not a lot better.

As this stage it is far better to convert the result to a SymPy matrix

In [22]:
from sympy import Matrix
Matrix(T.A)

Matrix([
[(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5), -(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5), -((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4),  0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)],
[(((-sin(q_0)*sin(q_1)*sin(q_2) + sin(q_0)*cos(q_1)*cos(q_2))

which is decently pretty printed by SymPy.

Often after a round of symbolic calculations there are simplifications that can be achieved. We can symbolically simplify each element of the `SE3` object by

In [23]:
Ts = T.simplify()

M = Matrix(Ts.A)
M

Matrix([
[-((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*cos(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*sin(q_5),   ((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*sin(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*cos(q_5),  (sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*sin(q_4) - sin(q_1 + q_2)*cos(q_0)*cos(q_4),  0.15005*sin(q_0) - 0.4318*sin(q_1 + q_2)*cos(q_0) + 0.4318*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1 + q_2)],
[ ((sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) - sin(q_0)*sin(q_4)*sin(q_1 + q_2))*cos(q_5) - (sin(q_0)*sin(q_3)*cos(q_1 + q_2) - cos(q_0)*cos(q_3))*sin(q_5), (-(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) + sin(q_0)*sin(q_4)*sin(q_1 + q_2))*sin(q_5) + (-sin(q_0)*sin(q_3)*cos(q_1 + q_2) + cos(q_0)*cos(q_3))*cos(q_5), -(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*sin(q

which is more compact (it takes a few seconds to compute). We can see that a trigometric _sum of angles_  substition has been performed, there are instances of sine and cosine of $q_1 + q_2$, the shoulder and elbow joints. This is to be expected since these joints are adjacent and have with parallel axes.

We can _slice_ the end-effector translationfrom the SymPy matrix

In [24]:
M[:3,3]

Matrix([
[ 0.15005*sin(q_0) - 0.4318*sin(q_1 + q_2)*cos(q_0) + 0.4318*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1 + q_2)],
[-0.4318*sin(q_0)*sin(q_1 + q_2) + 0.4318*sin(q_0)*cos(q_1) + 0.0203*sin(q_0)*cos(q_1 + q_2) - 0.15005*cos(q_0)],
[                                       0.4318*sin(q_1) + 0.0203*sin(q_1 + q_2) + 0.4318*cos(q_1 + q_2) + 0.672]])

The floating point constants here have been inherited from the kinematic model we imported at [this step](#Robot-forward-kinematics).  It would be possible to replace the non-zero kinematic constants $a_i$ and $d_i$ with symbols created using `sym.symbol` as shown in [this section](#Creating-a-symbolic-variable).

## Code generation
We can now use some of SymPy's superpowers to turn our forward kinematic expression into code

In [25]:
from sympy import ccode, pycode, octave_code
print(ccode(M, assign_to="T"))

T[0] = -((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*cos(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*sin(q_5);
T[1] = ((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*sin(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*cos(q_5);
T[2] = (sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*sin(q_4) - sin(q_1 + q_2)*cos(q_0)*cos(q_4);
T[3] = 0.15004999999999999*sin(q_0) - 0.43180000000000002*sin(q_1 + q_2)*cos(q_0) + 0.43180000000000002*cos(q_0)*cos(q_1) + 0.020299999999999999*cos(q_0)*cos(q_1 + q_2);
T[4] = ((sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) - sin(q_0)*sin(q_4)*sin(q_1 + q_2))*cos(q_5) - (sin(q_0)*sin(q_3)*cos(q_1 + q_2) - cos(q_0)*cos(q_3))*sin(q_5);
T[5] = (-(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) + sin(q_0)*sin(q_4)*sin(q_1 + q_2))*sin(q_5) + (-sin(q_0)*sin(q_3)*cos(q_1 + q_2) + cos(q_0)*cos(

which is pure C code that does not require any linear algebra package to compute. We simply need to define the values of `q_0` to `q_5` in order to determine the end-effector pose. The result is computed using symbolically simplified expressions.  The code is not optimized, but we could expect the compiler to perform some additional simplification.

The equivalent MATLAB code is

In [26]:
octave_code(M)

'[-((sin(q_0).*sin(q_3) - cos(q_0).*cos(q_3).*cos(q_1 + q_2)).*cos(q_4) + sin(q_4).*sin(q_1 + q_2).*cos(q_0)).*cos(q_5) - (sin(q_0).*cos(q_3) + sin(q_3).*cos(q_0).*cos(q_1 + q_2)).*sin(q_5) ((sin(q_0).*sin(q_3) - cos(q_0).*cos(q_3).*cos(q_1 + q_2)).*cos(q_4) + sin(q_4).*sin(q_1 + q_2).*cos(q_0)).*sin(q_5) - (sin(q_0).*cos(q_3) + sin(q_3).*cos(q_0).*cos(q_1 + q_2)).*cos(q_5) (sin(q_0).*sin(q_3) - cos(q_0).*cos(q_3).*cos(q_1 + q_2)).*sin(q_4) - sin(q_1 + q_2).*cos(q_0).*cos(q_4) 0.15005*sin(q_0) - 0.4318*sin(q_1 + q_2).*cos(q_0) + 0.4318*cos(q_0).*cos(q_1) + 0.0203*cos(q_0).*cos(q_1 + q_2); ((sin(q_0).*cos(q_3).*cos(q_1 + q_2) + sin(q_3).*cos(q_0)).*cos(q_4) - sin(q_0).*sin(q_4).*sin(q_1 + q_2)).*cos(q_5) - (sin(q_0).*sin(q_3).*cos(q_1 + q_2) - cos(q_0).*cos(q_3)).*sin(q_5) (-(sin(q_0).*cos(q_3).*cos(q_1 + q_2) + sin(q_3).*cos(q_0)).*cos(q_4) + sin(q_0).*sin(q_4).*sin(q_1 + q_2)).*sin(q_5) + (-sin(q_0).*sin(q_3).*cos(q_1 + q_2) + cos(q_0).*cos(q_3)).*cos(q_5) -(sin(q_0).*cos(q_3).*cos(q_

We can also output Python code

In [27]:
print(pycode(M))

ImmutableDenseMatrix([[-((math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.cos(q_4) + math.sin(q_4)*math.sin(q_1 + q_2)*math.cos(q_0))*math.cos(q_5) - (math.sin(q_0)*math.cos(q_3) + math.sin(q_3)*math.cos(q_0)*math.cos(q_1 + q_2))*math.sin(q_5), ((math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.cos(q_4) + math.sin(q_4)*math.sin(q_1 + q_2)*math.cos(q_0))*math.sin(q_5) - (math.sin(q_0)*math.cos(q_3) + math.sin(q_3)*math.cos(q_0)*math.cos(q_1 + q_2))*math.cos(q_5), (math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.sin(q_4) - math.sin(q_1 + q_2)*math.cos(q_0)*math.cos(q_4), 0.15005*math.sin(q_0) - 0.4318*math.sin(q_1 + q_2)*math.cos(q_0) + 0.4318*math.cos(q_0)*math.cos(q_1) + 0.0203*math.cos(q_0)*math.cos(q_1 + q_2)], [((math.sin(q_0)*math.cos(q_3)*math.cos(q_1 + q_2) + math.sin(q_3)*math.cos(q_0))*math.cos(q_4) - math.sin(q_0)*math.sin(q_4)*math.sin(q_1 + q_2))*math.cos(q_5) - (math.s

Which constructs an instance of a SymPy `ImmutableDenseMatrix` which we can turn into a function

In [28]:
from sympy import lambdify
T_func = lambdify(q, M, modules='numpy')

If we pass in the zero joint angles we get the familiar forward kinematic solution result

In [29]:
T_func(0, 0, 0, 0, 0, 0)

array([[ 1.     , -0.     , -0.     ,  0.4521 ],
       [ 0.     ,  1.     , -0.     , -0.15005],
       [ 0.     , -0.     ,  1.     ,  1.1038 ],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

SymPy also supports printing code in C++ (`cxxcode`), Fortran (`fcode`), JavaScript (`jscode`), Julia (`julia_code`), Rust (`rust_code`), Mathematica (`mathematica_code`) and Octave/MATLAB (`octave_code`).

The [SymPy autowrap](https://docs.sympy.org/latest/modules/codegen.html#autowrap) capability automatically generates code, writes it to disk, compiles it, and imports it into the current session.  It creates a wrapper using Cython and creates a numerical function.